# Vowpal Wabbit в NLP
Автоматическая обработка текстов - 2017, семинар 4.

В этом семинаре мы познакомимся с библиотекой Vowpal Wabbit и решим с его помощью задачу многоклассовой классификации на больших данных. 
Данные скачайте [здесь](https://www.kaggle.com/c/predict-closed-questions-on-stack-overflow/data) или запустите следующие две ячейки.

In [ ]:
! wget https://www.dropbox.com/s/r0q0p0uprhcp8bb/train-sample.zip
! unzip train-sample.zip

In [ ]:
! wget https://www.dropbox.com/s/50vw2gsglc91f6o/train.zip
! unzip train.zip

Чтобы на семинаре не тратить время на обработку и обучение моделей на всех данных, предлагается использовать только небольшую подвыборку (`train-sample.csv`). Но сдавать ноутбук все равно необходимо с результатами на **всех** данных.

In [ ]:
import csv

INPUT_DATA = 'train.csv'
# INPUT_DATA = 'train-sample.csv'

reader = csv.DictReader(open(INPUT_DATA))
dict(next(reader))

Каждый объект выборки соответствует некоторому посту на Stack Overflow. Требуется построить модель, определяющую статус поста. Подробнее про задачу и формат данных можно прочитать на [странице соревнования](https://www.kaggle.com/c/predict-closed-questions-on-stack-overflow).

Перед обучением модели из Vowpal Wabbit данные следует сохранить в специальный формат: <br>
`label |namespace1 feature1:value1 feature2 feature3:value3 |namespace2 ...` <br>
Записи `feature` и `feature:1.0` эквивалентны. Выделение признаков в смысловые подгруппы (namespaces) позволяет создавать взаимодействия между ними. Подробнее про формат входных данных можно прочитать [здесь](https://github.com/JohnLangford/vowpal_wabbit/wiki/Input-format).

Ниже реализована функция, которая извлекает признаки с помощью подаваемого на вход экстрактора, разбивает данные на трейн и тест и записывает их на диск.

In [ ]:
STATUSES = ['not a real question', 'not constructive', 'off topic', 'open', 'too localized']
STATUS_DICT = {status: i+1 for i, status in enumerate(STATUSES)}

def data2vw(features_extractor, train_output='train', test_output='test', ytest_output='ytest'):
    reader = csv.DictReader(open(INPUT_DATA))
    writer_train = open(train_output, 'w')
    writer_test = open(test_output, 'w')
    writer_ytest = open(ytest_output, 'w')
    
    for row in reader:
        label = STATUS_DICT[row['OpenStatus']]
        features = features_extractor(row)
        output_line = '%s %s\n' % (label, features)
        if int(row['PostId']) % 2 == 0:
            writer_train.write(output_line)
        else:
            writer_test.write(output_line)
            writer_ytest.write('%s\n' % label)
            
    writer_train.close()
    writer_test.close()
    writer_ytest.close()

Начнем с простейшей модели. В качестве признаков возьмите заголовки и очистите их: приведите символы к нижнему регистру, удалите пунктуацию. Также приветствуется использование стеммеров/лемматизаторов, однако учтите, что они могут сильно замедлить скорость обработки.

In [ ]:
import re

def extract_title(row):
    title = row['Title']
    # YOUR CODE HERE


data2vw(lambda row: '| %s' % extract_title(row))

! head -n 5 train

Обучим `vw` модель. Параметр `-d` отвечает за путь к обучающей выборке, `-f` – за путь к модели, `--oaa` – за режим мультиклассовой классификации `one-against-all`. Подробное описание всех параметров можно найти [здесь](https://github.com/JohnLangford/vowpal_wabbit/wiki/Command-line-arguments) или вызвав `vw --help`.

In [ ]:
! vw -d train --loss_function logistic --oaa 5 -f model

Применим модель к тестовой выборке и сохраним предсказания.

In [ ]:
! vw -i model -t test -r pred

In [ ]:
! head -n 3 pred

Реализуйте функцию, которая вычисляет `logloss` и `accuracy`, не загружая вектора в память. Используйте `softmax`, чтобы получить вероятности.

In [ ]:
def get_scores(ytest_input='ytest', pred_input='pred'):
    n, error, loss = 0, 0, 0
    reader_ytest = open(ytest_input, 'r')
    reader_pred = open(pred_input, 'r')
    
    for label, pred in zip(reader_ytest, reader_pred):
        # YOUR CODE HERE
        
    reader_ytest.close()
    reader_pred.close()
    return - loss / n, 1 - float(error) / n

In [ ]:
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

На оригинальных данных `logloss` должен быть меньше `0.20`, `accuracy` больше `0.95`. Если это не так, то скорее всего у вас ошибка.

Теперь попробуем улучшить модель, добавив новые признаки, порождаемые словами. В `vowpal wabbit` есть возможность делать это прямо на лету. Воспользуйтесь параметрами `affix`, `ngram`, `skips`.

Далее везде при подборе параметров ориентируйтесь на улучшение `logloss`. Используйте `--quiet` или `-P`, чтобы избавиться от длинных выводов при обучении и применении моделей.

In [ ]:
# YOUR CODE HERE

Часто качество `vw` модели получается учушить увеличением числа проходов по обучающей выборке (параметр `--passes`) и увеличением числа бит хэш-функции для уменьшения числа коллизий признаков (параметр `-b`). Подробнее про то, где в `vowpal wabbit` используется хэш-функция, можно прочитать [здесь](https://github.com/JohnLangford/vowpal_wabbit/wiki/Feature-Hashing-and-Extraction). Как меняется качество при изменении этих параметров? Верно ли, что при увеличении значений параметров `--passes` и `-b` качество всегда не убывает и почему?

In [ ]:
# YOUR CODE HERE

Теперь интерес представляет то, какие признаки оказались наиболее важными для модели. Для этого сначала переведем модель в читаемый формат.

In [ ]:
! vw -i model -t --invert_hash model.readable train --quiet
! head -n 30 model.readable

Первые несколько строк соответствуют информации о модели. Далее следуют строчки вида `feature[label]:hash:weight`. Выделите для каждого класса 10 признаков с наибольшими по модулю весами. Постарайтесь сделать ваш алгоритм прохода по файлу константным по памяти. Например, можно воспользоваться [кучей](https://docs.python.org/2/library/heapq.html).

In [ ]:
# YOUR CODE HERE

Добавим признаки, извлеченные из текста поста (поле `BodyMarkdown`). В этом поле находится более подробная информация о вопросе, и часто туда помещают код, формулы и т.д. При удалении пунктуации мы потеряем много полезной информации, однако модель "мешка слов" на сырых данных может сильно раздуть признаковое пространство. В таких случаях работают с n-граммами на символах. <br>
Будьте осторожны: символы "`:`" и "`|`" нельзя использовать в названиях признаков, поскольку они являются служебными для `vw`-формата. Замените эти символы на два других редко встречающихся в выборке (или вообще не встречающихся). Также не забудьте про "`\n`". <br>
Поскольку для каждого документа одна n-грамма может встретиться далеко не один раз, то будет экономнее записывать признаки в формате `[n-грамма]:[число вхождений]`.

Также добавьте тэги (поля вид `TagN`). Приветствуется добавление информации о пользователе из других полей. Только не используйте `PostClosedDate` – в нем содержится информация о таргете.

In [ ]:
def extract_ngram_body(row, ngram=3):
    # YOUR CODE HERE


def extract_tags(row):
    # YOUR CODE HERE

Объединим все вместе. Реализуйте экстрактор признаков, который выделяет каждую подгруппу в отдельный namespace.

In [ ]:
extractors_list = [
    ('t', extract_title), 
    ('b', extract_ngram_body), 
    ('a', extract_tags)
] # (namespace, extractor)


def make_feature_extractor(extractors_list):
    def feature_extractor(row):
        # YOUR CODE HERE
    return feature_extractor
    
    
data2vw(make_feature_extractor(extractors_list))

! head -n 1 train

Поэкспериментируйте с другими параметрами модели. Добавьте квадратичные взаимодействия между различными блоками признаков, измените параметры оптимизатора, добавьте регуляризацию и т.д. Совсем необязательно перебирать все параметры по сетке и добиваться оптимального качества. Для нас важнее то, насколько хорошо вы разобрались с возможностями библиотеки и продемонстрировали это.

Выберите не менее трех параметров. Для каждого из них объясните, почему по вашему мнению его изменение может улучшить качество модели, подберите оптимальное значение. Можете перебрать несколько значений "руками", а можете воспользоваться [vw-hypersearch](https://github.com/JohnLangford/vowpal_wabbit/wiki/Using-vw-hypersearch) или [vw-hyperopt](https://github.com/JohnLangford/vowpal_wabbit/blob/master/utl/vw-hyperopt.py) ([статья на хабре](https://habrahabr.ru/company/dca/blog/272697/)). Какие параметры повлияли на улучшение качества сильнее всего?

In [ ]:
# YOUR CODE HERE